In [1]:
import numpy as np
import pandas as pd

In [2]:
ppi_data = pd.read_csv("2023-02-17_DHFR-PCA_HRR25_Cterm_InteractorsFiltered.csv")

In [3]:
ppi_data.head()

,Miniarray.ORF,Construct_ID,mean,sd,N,Construct,GeneID,MSA_order,Phylo_order,Array_order,Species,Gene,Pos,Species_position,Miniarray.Gene,Protein.Removed
0,AAT2,pKB11.12.YLR027C,51992.75,7006.367541,2,pKB11,12,6,21,84,T.delbrueckii,TDEL0E05230,Anc,T.phaffii(A),YLR027C,0
1,AAT2,pKB11.24.YLR027C,11554.25,3986.314479,2,pKB11,24,24,25,84,H. sapiens,CSNK1A1L,NaN,H. sapiens(CSNK1A1L),YLR027C,0
2,AAT2,pKB11.14.YLR027C,29038.75,3331.533600,2,pKB11,14,14,14,84,V.polyspora,Kpol_1002.53,A,N.castellii(A),YLR027C,0
3,AAT2,pKB11.1.YLR027C,25430.25,3173.141681,2,pKB11,1,1,2,84,S.bayanusvar.uvarum,Suva_16.102,A,S.bayanusvar.uvarum(A),YLR027C,0
4,AAT2,pKB11.22.YLR027C,3486.50,1021.769299,2,pKB11,22,22,23,84,H. sapiens,CSNK1A1,NaN,H. sapiens(CSNK1A1),YLR027C,0


In [4]:
d = {}

for index, row in ppi_data.iterrows():
    orf      = row['Miniarray.ORF']
    gene     = row['Miniarray.Gene']
    auc      = row['mean']
    species  = row['Species']
    ortholog = row['Gene']
    pos      = row['Pos']
    
    if (orf,gene) not in d:
        d[(orf,gene)] = {}
    try:
        d[(orf,gene)][species][(ortholog,pos)] = {"PPI.AUC":auc     }
    except KeyError:
        d[(orf,gene)][species] = {(ortholog,pos):{"PPI.AUC":auc  }   }

    
    
        
    
    


In [5]:
out_ll = [["PPI_partner.Gene","PPI_partner.ORF","Ortholog.Species","State","PPI.NA","Ortholog.GeneA","Ortholog.GeneB","Ortholog.GeneAnc","PPI.Ortholog.GeneA","PPI.Ortholog.GeneB","PPI.Ortholog.Anc","PPI.A_div_B","PPI.Scer","PPI.Vehicle"]]


In [6]:
dup_state = {

"S.bayanusvar.uvarum" : "A",
"S.kudriavzevii": "A",
"S.mikatae": "A",
"C.glabrata": "B",
"S.cerevisiae":"A",
"Z.rouxii":"Anc",
"T.delbrueckii":"Anc",
"N.castellii":"AB",
"V.polyspora":"AB",
"T.phaffii":"AB",
"T.blattae":"AB",
"N.dairenensis":"AB",
"K.africana":"AB",
"K.naganishii":"AB",
"Vehicle":"Vehicle"}


In [7]:
for ppi_partner in d:
    #print(ppi_partner)
    orf  = ppi_partner[0]
    gene = ppi_partner[1]
    
    # Get value for Scer for adding as control
    if "S.cerevisiae" in d[ppi_partner]:
        for k in d[ppi_partner]["S.cerevisiae"]:
            for ppi_k in d[ppi_partner]["S.cerevisiae"][k]:
                scer_v = d[ppi_partner]["S.cerevisiae"][k][ppi_k]
    else:
        scer_v = "NA"
    if "Vehicle" in d[ppi_partner]:
        for k in d[ppi_partner]["Vehicle"]:
            for ppi_k in d[ppi_partner]["Vehicle"][k]:
                vehicle_v = d[ppi_partner]["Vehicle"][k][ppi_k]
    else:
        vehicle_v = "NA"

    for species in d[ppi_partner]:
        if species != "H. sapiens":
            state = dup_state[species]

            a   = [i for i in d[ppi_partner][species] if i[1] =="A"] 
            b   = [i for i in d[ppi_partner][species] if i[1] =="B"] 
            anc = [i for i in d[ppi_partner][species] if i[1] == "Anc"]
            nan = [i for i in d[ppi_partner][species] if str(i[1]) == False]
        
            try:
                ppi_anc = d[ppi_partner][species][anc[0]]["PPI.AUC"]
            except IndexError:
                ppi_anc = ""
                
            try:
                ppi_a = float(d[ppi_partner][species][a[0]]["PPI.AUC"])+1
            except IndexError:
                ppi_a = ""
            try:
                ppi_b = float(d[ppi_partner][species][b[0]]["PPI.AUC"])+1
            except IndexError:
                ppi_b = ""
                


            if (ppi_a != "") & (ppi_b != ""):
                ppi_AdivB = ppi_a/ppi_b
                missing_val = "No"           
                
            elif ppi_anc != "":
                ppi_AdivB = 1.0
                missing_val = "No"           
                
            elif ppi_a != "":
                ppi_AdivB  = ppi_a/1.0
                if state =="AB":
                    missing_val = "B"      
                
            elif ppi_b != "":
                ppi_AdivB = ppi_AdivB = 1.0/ppi_b
                if state =="AB":
                    missing_val = "A"      
            else:
                ppi_AdivB = 1.0
                missing_val = "No" 
                
            anc += nan
            a.append(("",""))
            b.append(("",""))
            anc.append(("",""))
            if state =="Vehicle":
                l = [gene,orf,species,state,missing_val,"Vehicle","Vehicle","Vehicle",vehicle_v,vehicle_v,vehicle_v,1.0,scer_v,vehicle_v  ]
            else:
                l = [gene,orf,species,state,missing_val,a[0][0],b[0][0],anc[0][0],ppi_a,ppi_b,ppi_anc,ppi_AdivB,scer_v,vehicle_v  ]
                
            out_ll.append(l)
        
    

In [141]:
def LL2csv(LL,name):
    with open(name,"w") as F:
        for L in LL:
            F.write("%s\n" %  (",").join( [str(i) for i in L]))
    # This script outputs dict as string, so do not print anything
    print("Generated : %s"%(name))
    F.close()

In [142]:
LL2csv(out_ll,"20230614_parsed_PPI_data.csv")

Generated : 20230614_parsed_PPI_data.csv
